In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from nb_005 import *

# STL-10

## Basic data aug

In [ ]:
PATH = Path('data/stl10')

In [ ]:
data_mean, data_std = map(tensor, ([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]))
data_norm,data_denorm = normalize_funcs(data_mean,data_std)

In [ ]:
train_ds = FilesDataset.from_folder(PATH/'train')
valid_ds = FilesDataset.from_folder(PATH/'valid')

In [ ]:
x=Image(valid_ds[2][0])
x.show()
x.shape

In [ ]:
size=96
tfms = get_transforms(do_flip=True, max_rotate=5, max_zoom=1.25, max_lighting=0.4, max_warp=0.15)
# tfms = get_transforms(do_flip=True)#, max_rotate=5, max_lighting=0.2)
tds = transform_datasets(train_ds, valid_ds, tfms, size=size)
data = DataBunch(*tds, bs=32, num_workers=8, tfms=data_norm)

In [ ]:
(x,y) = next(iter(data.valid_dl))

_,axs = plt.subplots(4,4,figsize=(12,12))
for i,ax in enumerate(axs.flatten()): show_image(data_denorm(x[i].cpu()), ax)

In [ ]:
_,axs = plt.subplots(4,4,figsize=(12,12))
for ax in axs.flat: show_image(tds[0][1][0], ax)

## Train

In [ ]:
from torchvision.models import resnet18, resnet34, resnet50
arch = resnet50

In [ ]:
learn = ConvLearner(data, arch, 2, wd=1e-3, callback_fns=[BnFreeze], opt_fn=optim.Adam, dps=(0.1,0.2))
learn.metrics = [accuracy]
learn.split(lambda m: (m[0][6], m[1]))
learn.freeze()

In [ ]:
lr_find(learn)
learn.recorder.plot()

In [ ]:
lr = 1e-3

In [ ]:
fit_one_cycle(learn, lr, 3, moms=(0.9,0.9))

In [ ]:
learn.save('0')

## Gradual unfreezing

In [ ]:
learn.load('0')

In [ ]:
learn.freeze_to(1)

In [ ]:
lrs = np.array([lr/9, lr/3, lr])

In [ ]:
fit_one_cycle(learn, lrs/3, 3)

In [ ]:
learn.save('1')

In [ ]:
learn.load('1')

In [ ]:
learn.unfreeze()

In [ ]:
fit_one_cycle(learn, lrs/9, 3)

In [ ]:
learn.save('2')

In [ ]:
# TODO remove layer groups; use start_layer / start_lr

## TTA

In [ ]:
def transform_datasets(train_ds, valid_ds, tfms, size=None):
    return (DatasetTfm(train_ds, tfms[0], size=size),
            DatasetTfm(valid_ds, tfms[1], size=size),
            DatasetTfm(valid_ds, tfms[0], size=size))

In [ ]:
class DataBunch():
    def __init__(self, train_ds, valid_ds, augm_ds, bs=64, device=None, num_workers=4, **kwargs):
        self.device = default_device if device is None else device
        self.train_dl = DeviceDataLoader.create(train_ds, bs,   shuffle=True,  num_workers=num_workers, **kwargs)
        self.valid_dl = DeviceDataLoader.create(valid_ds, bs*2, shuffle=False, num_workers=num_workers, **kwargs)
        self.augm_dl  = DeviceDataLoader.create(augm_ds,  bs*2, shuffle=False, num_workers=num_workers, **kwargs)

    @classmethod
    def create(cls, train_ds, valid_ds, train_tfm=None, valid_tfm=None, dl_tfms=None, **kwargs):
        return cls(DatasetTfm(train_ds, train_tfm), DatasetTfm(valid_ds, valid_tfm), DatasetTfm(valid_ds, train_tfm), 
                   tfms=dl_tfms, **kwargs)

    @property
    def train_ds(self): return self.train_dl.dl.dataset
    @property
    def valid_ds(self): return self.valid_dl.dl.dataset

In [ ]:
tds = transform_datasets(train_ds, valid_ds, tfms, size=size)
data = DataBunch(*tds, num_workers=8, tfms=data_norm)

In [ ]:
learn = ConvLearner(data, arch, 2, wd=1e-3)
learn.metrics = [accuracy]

In [ ]:
learn.load('2')

In [ ]:
_,axs = plt.subplots(2,4,figsize=(12,6))
for ax in axs.flat: show_image(tds[2][1][0], ax)

In [ ]:
model = learn.model
model.eval();

In [ ]:
with torch.no_grad():
    preds,y = zip(*[(model(xb.detach()), yb.detach()) for xb,yb in data.valid_dl])

preds = torch.cat(preds)
y = torch.cat(y)

In [ ]:
accuracy(preds, y)

In [ ]:
def get_preds(model, dl):
    with torch.no_grad():
        return torch.cat([model(xb.detach()) for xb,yb in dl])

In [ ]:
all_preds = torch.stack([get_preds(model, data.augm_dl) for _ in range(4)])

In [ ]:
avg_preds = all_preds.mean(0)
avg_preds.shape

In [ ]:
accuracy(avg_preds, y)

In [ ]:
beta=0.5
accuracy(preds*beta + avg_preds*(1-beta), y)

## Fin